In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd
import sys
sys.path.append('../DNN')
from neuralnet import NeuralNetwork, Dataset
from optimizer import Optimizer
from losses import BinaryCrossEntropy
from metrics import accuracy
from activation import ReLUActivation, SigmoidActivation
from layers import DenseLayer, DropoutLayer
import pickle

# Caminhos dos arquivos (na mesma pasta)
MODEL_PATH = "../models/dnn.pkl"
INPUT_FILE = "dataset2_inputs.csv"
OUTPUT_FILE = "results/submissao1-grupo011-s1.csv"

# Ler os novos dados de entrada, extraindo IDs e textos
ids = []
texts = []
with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    header = lines[0].strip()
    if 'ID' not in header or 'Text' not in header:
        print(f"Erro: Cabeçalho deve conter 'ID' e 'Text' em {INPUT_FILE}")
        exit(1)
    for line in lines[1:]:
        parts = line.split(';', 1)  # Dividir no primeiro ';'
        if len(parts) > 1:
            id_part = parts[0].strip()  # ID é a parte antes do primeiro ';'
            text = parts[1].strip()     # Texto é tudo após o primeiro ';'
            ids.append(id_part)
            texts.append(text)
        else:
            ids.append('')
            texts.append('')

# Criar o modelo básico sem adicionar camadas ainda
net = NeuralNetwork(
    epochs=10,
    batch_size=16,
    learning_rate=0.005,
    optimizer=Optimizer(learning_rate=0.005, momentum=0.9, weight_decay=1e-5),
    loss=BinaryCrossEntropy,
    metric=accuracy
)

# Carregar o modelo e o vectorizer
try:
    vectorizer = net.load(MODEL_PATH)
    if vectorizer is None:
        print("Erro: Vectorizer não encontrado no arquivo do modelo.")
        exit(1)
except FileNotFoundError:
    print(f"Erro: Arquivo {MODEL_PATH} não encontrado.")
    exit(1)
except Exception as e:
    print(f"Erro ao carregar o modelo: {e}")
    exit(1)

# Transformar os novos dados com o vectorizer carregado
X = vectorizer.transform(texts).toarray()  # Apenas transformar, usando o vectorizer do treino

# Adicionar as camadas agora que sabemos o input_shape
n_features = X.shape[1]
net.add(DenseLayer(32, (n_features,)))
net.add(ReLUActivation())
net.add(DropoutLayer(0.3))
net.add(DenseLayer(16))
net.add(ReLUActivation())
net.add(DropoutLayer(0.4))
net.add(DenseLayer(1))
net.add(SigmoidActivation())

# Fazer previsões
input_dataset = Dataset(X, None)
predictions = net.predict(input_dataset)
predictions_classes = np.round(predictions).astype(int)
predictions_labels = ['IA' if pred == 1 else 'Human' for pred in predictions_classes.flatten()]

# Criar DataFrame com IDs e previsões
output_df = pd.DataFrame({
    'ID': ids,
    'Label': predictions_labels
})
output_df.to_csv(OUTPUT_FILE, index=False)
print(f"Previsões salvas em {OUTPUT_FILE} com IDs")

Model loaded successfully from ../models/dnn.pkl
Previsões salvas em results/submissao1-grupo011-s1.csv com IDs


In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd
import sys
sys.path.append('../RNN')
from rnn import RecurrentNeuralNetwork  # Import the custom RNN model
from optimizer import Optimizer  # Optimizer used for training
from losses import BinaryCrossEntropy  # Loss function
from metrics import accuracy  # Metric for evaluation
from activation import ReLUActivation, SigmoidActivation  # Activation functions
from layers import RecurrentLayer, DropoutLayer, DenseLayer  # RNN layers
import pickle

# Caminhos dos arquivos (na mesma pasta)
MODEL_PATH = "../models/rnn.pkl"  # Updated model path for RNN
INPUT_FILE = "dataset2_inputs.csv"  # Input dataset
OUTPUT_FILE = "results/submissao1-grupo011-s2.csv"  # Output predictions

# Ler os novos dados de entrada, extraindo IDs e textos
ids = []
texts = []
with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    header = lines[0].strip()
    if 'ID' not in header or 'Text' not in header:
        print(f"Erro: Cabeçalho deve conter 'ID' e 'Text' em {INPUT_FILE}")
        exit(1)
    for line in lines[1:]:
        parts = line.split(';', 1)  # Dividir no primeiro ';'
        if len(parts) > 1:
            id_part = parts[0].strip()  # ID é a parte antes do primeiro ';'
            text = parts[1].strip()     # Texto é tudo após o primeiro ';'
            ids.append(id_part)
            texts.append(text)
        else:
            ids.append('')
            texts.append('')

# Carregar o modelo e o vectorizer
try:
    rnn = RecurrentNeuralNetwork(
        epochs=10,
        batch_size=16,
        learning_rate=0.005,
        optimizer=Optimizer(learning_rate=0.005, momentum=0.9, weight_decay=1e-5),
        loss=BinaryCrossEntropy,
        metric=accuracy
    )
    
    vectorizer = rnn.load(MODEL_PATH)
    if vectorizer is None:
        print("Erro: Vectorizer não encontrado no arquivo do modelo.")
        exit(1)
except FileNotFoundError:
    print(f"Erro: Arquivo {MODEL_PATH} não encontrado.")
    exit(1)
except Exception as e:
    print(f"Erro ao carregar o modelo: {e}")
    exit(1)

# Transformar os novos dados com o vectorizer carregado
X = vectorizer.transform(texts).toarray()
X = X.reshape((X.shape[0], X.shape[1], 1))  # Redimensionar para sequência (samples, time_steps, features)

# Adicionar camadas RNN
n_features = X.shape[2]
rnn.add(RecurrentLayer(32, (n_features,)))  # Camada recorrente
rnn.add(DropoutLayer(0.3))  # Dropout layer
rnn.add(DenseLayer(16))  # Camada densa após a camada RNN
rnn.add(ReLUActivation())  # ReLU Activation
rnn.add(DropoutLayer(0.4))
rnn.add(DenseLayer(1))  # Saída final
rnn.add(SigmoidActivation())  # Sigmoid activation para classificação binária

# Fazer previsões
input_dataset = Dataset(X, None)
predictions = rnn.predict(input_dataset)
predictions_classes = np.round(predictions).astype(int)
predictions_labels = ['IA' if pred == 1 else 'Human' for pred in predictions_classes.flatten()]

# Criar DataFrame com IDs e previsões
output_df = pd.DataFrame({
    'ID': ids,
    'Label': predictions_labels
})

# Salvar as previsões em um arquivo CSV
output_df.to_csv(OUTPUT_FILE, index=False)
print(f"Previsões salvas em {OUTPUT_FILE} com IDs")


Model loaded successfully from ../models/rnn.pkl
Previsões salvas em results/submissao1-grupo011-s2.csv com IDs
